In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Settings:
pd.set_option('display.width', 190)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)
pd.set_option('max_colwidth', 200)
pd.options.display.float_format = '{:.4f}'.format
plt.style.use('default')
np.set_printoptions(threshold = 30, edgeitems = 30, precision = 2, suppress = False)

In [3]:
df = pd.read_csv("../original_data/BCI-values.csv")
df.Date = pd.to_datetime(df.Date, dayfirst=True)
df = df.set_index("Date")
# log transformation
df.BCI = df.BCI.apply(lambda x: np.log(x))
df.BCIg = df.BCIg.apply(lambda x: np.log(x + abs(min(df.BCIg)) + 0.1)) # Because there are values that are negative, we transform all values just a little bit positive to be able to make a logarithmic transformation.
df.BCIp = df.BCIp.apply(lambda x: np.log(x + abs(min(df.BCIp)) + 0.1))

target = pd.read_csv("../original_data/USRECD.csv")
target = target.rename({"DATE": "Date"}, axis=1)
target.Date = pd.to_datetime(target.Date)
df = pd.merge(df, target, on="Date")
df.head()


,Date,BCI,BCIp,BCIg,USRECD
0,1967-02-09,4.6052,6.5870,3.4751,0
1,1967-02-16,4.6052,6.5863,3.4751,0
2,1967-02-23,4.6012,6.5774,3.4751,0
3,1967-03-02,4.6032,6.5820,3.4751,0
4,1967-03-09,4.6042,6.5852,3.4751,0


In [4]:
ie = pd.read_csv("../original_data/Yale IE.csv")
ie.columns

Index(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', '  Consumer', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Real', 'Unnamed: 10', 'Real.1', 'Earnings',
       'Unnamed: 13', 'Earnings.1', 'Unnamed: 15', 'Unnamed: 16', 'Monthly', 'Real.2', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24',
       'Unnamed: 25'],
      dtype='object')

In [5]:
ie.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Consumer,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Real,Unnamed: 10,Real.1,Earnings,Unnamed: 13,Earnings.1,Unnamed: 15,Unnamed: 16,Monthly,Real.2,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,NaN,S&P,NaN,NaN,Price,NaN,Long,NaN,NaN,Total,NaN,TR,Ratio,NaN,Ratio,NaN,Excess,Total,Total,10 Year,10 Year,Real 10 Year,NaN,NaN,NaN,NaN
1,NaN,Comp.,Dividend,Earnings,Index,Date,Interest,Real,Real,Return,Real,Scaled,P/E10 or,NaN,TR P/E10 or,NaN,CAPE,Bond,Bond,Annualized Stock,Annualized Bonds,Excess Annualized,NaN,NaN,NaN,NaN
2,Date,P,D,E,CPI,Fraction,Rate GS10,Price,Dividend,Price,Earnings,Earnings,CAPE,NaN,TR CAPE,NaN,Yield,Returns,Returns,Real Return,Real Return,Returns,NaN,NaN,NaN,NaN
3,1871.01,4.44,0.26,0.40,12.46,1871.04,5.32,97.25,5.69,97.25,8.76,8.76,NaN,NaN,NaN,NaN,NaN,1.00,1.00,13.06%,9.25%,3.81%,NaN,NaN,NaN,NaN
4,1871.02,4.50,0.26,0.40,12.84,1871.13,5.32,95.64,5.53,96.10,8.50,8.54,NaN,NaN,NaN,NaN,NaN,1.00,0.97,13.09%,9.46%,3.62%,NaN,NaN,NaN,NaN


In [6]:
ie["Unnamed: 19"]

0                10 Year
1       Annualized Stock
2            Real Return
3                 13.06%
4                 13.09%
              ...       
1808                 NaN
1809                 NaN
1810                 NaN
1811                 NaN
1812                 NaN
Name: Unnamed: 19, Length: 1813, dtype: object

In [7]:
ie = pd.read_csv("../original_data/Yale IE.csv")
# drop "garbage"
ie = ie.drop(index=[0, 1, 2, 1812])
ie = ie.rename({"Unnamed: 0": "Date", "Unnamed: 1": "IE_SP_Comp", "Unnamed: 2": "IE_SP_Dividend", "Unnamed: 3": "IE_SP_Earnings", "  Consumer": "IE_Consumer_CPI", 
                "Unnamed: 6": "IE_Long_Interest", "Unnamed: 7": "IE_Real_Price", "Unnamed: 8": "IE_Real_Dividend", "Real": "IE_Return_Price", "Unnamed: 10": "IE_Real_Earnings",
               "Real.1": "IE_Scaled_Earnings", "Monthly": "IE_Monthly_Returns", "Real.2": "IE_Real_Returns"}, axis=1)
new_features = ['IE_SP_Comp', 'IE_SP_Dividend', 'IE_SP_Earnings', 'IE_Consumer_CPI', 'IE_Long_Interest', 'IE_Real_Price', 'IE_Real_Dividend', 'IE_Return_Price', 'IE_Real_Earnings',
                'IE_Scaled_Earnings', 'IE_Monthly_Returns', 'IE_Real_Returns']

def remove_dot(x):
    if isinstance(x, str) and ',' in x:
        return x.replace(',', '')
    else:
        return x
    
def remove_percent(x):
    if '%' in x:
        return x.replace('%', '')
    else:
        return x

cleaners = {'IE_Return_Price': remove_dot, 'IE_Scaled_Earnings': remove_dot}

for feature in new_features:
    print(feature)
    
    if feature in cleaners.keys():
        ie[feature] = ie[feature].apply(cleaners[feature])
    ie[feature] = pd.to_numeric(ie[feature])
    # logorithm
    ie[feature] = ie[feature].apply(lambda x: np.log(x))

ie = ie.reset_index(drop=True)

# data cleaning
def cleaner(x):
    x = str(x)
    if x.endswith('.1'):
        return x.split('.')[0] + '.10'
    else:
        return x

ie.Date = ie.Date.apply(cleaner, convert_dtype=False)
ie.Date = pd.to_datetime(ie.Date)
# change to the correct time
ie.Date = ie.Date + pd.DateOffset(months=1, days=-1)
ie.head()

IE_SP_Comp
IE_SP_Dividend
IE_SP_Earnings
IE_Consumer_CPI
IE_Long_Interest
IE_Real_Price
IE_Real_Dividend
IE_Return_Price
IE_Real_Earnings
IE_Scaled_Earnings
IE_Monthly_Returns
IE_Real_Returns


,Date,IE_SP_Comp,IE_SP_Dividend,IE_SP_Earnings,IE_Consumer_CPI,Unnamed: 5,IE_Long_Interest,IE_Real_Price,IE_Real_Dividend,IE_Return_Price,IE_Real_Earnings,IE_Scaled_Earnings,Earnings,Unnamed: 13,Earnings.1,Unnamed: 15,Unnamed: 16,IE_Monthly_Returns,IE_Real_Returns,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,1871-01-31,1.4907,-1.3471,-0.9163,2.5225,1871.04,1.6715,4.5773,1.7387,4.5773,2.1702,2.1702,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,13.06%,9.25%,3.81%,NaN,NaN,NaN,NaN
1,1871-02-28,1.5041,-1.3471,-0.9163,2.5526,1871.13,1.6715,4.5606,1.7102,4.5654,2.1401,2.1448,NaN,NaN,NaN,NaN,NaN,0.0000,-0.0305,13.09%,9.46%,3.62%,NaN,NaN,NaN,NaN
2,1871-03-31,1.5282,-1.3471,-0.9163,2.5673,1871.21,1.6734,4.5701,1.6956,4.5795,2.1258,2.1353,NaN,NaN,NaN,NaN,NaN,0.0000,-0.0408,13.10%,9.62%,3.48%,NaN,NaN,NaN,NaN
3,1871-04-30,1.5560,-1.3471,-0.9163,2.5305,1871.29,1.6734,4.6350,1.7317,4.6491,2.1622,2.1770,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,12.21%,9.10%,3.11%,NaN,NaN,NaN,NaN
4,1871-05-31,1.5810,-1.3471,-0.9163,2.5072,1871.37,1.6734,4.6831,1.7544,4.7016,2.1861,2.2039,NaN,NaN,NaN,NaN,NaN,0.0000,0.0296,12.26%,8.95%,3.31%,NaN,NaN,NaN,NaN


In [8]:
for feature in new_features:
    df[feature] = df.Date.apply(lambda x: ie[ie.Date <= x].iloc[-1][feature])
df.head()

,Date,BCI,BCIp,BCIg,USRECD,IE_SP_Comp,IE_SP_Dividend,IE_SP_Earnings,IE_Consumer_CPI,IE_Long_Interest,IE_Real_Price,IE_Real_Dividend,IE_Return_Price,IE_Real_Earnings,IE_Scaled_Earnings,IE_Monthly_Returns,IE_Real_Returns
0,1967-02-09,4.6052,6.5870,3.4751,0,4.4362,1.0578,1.7084,3.4935,1.5217,6.5522,3.1739,11.4502,3.8238,8.7218,0.0000,2.5153
1,1967-02-16,4.6052,6.5863,3.4751,0,4.4362,1.0578,1.7084,3.4935,1.5217,6.5522,3.1739,11.4502,3.8238,8.7218,0.0000,2.5153
2,1967-02-23,4.6012,6.5774,3.4751,0,4.4362,1.0578,1.7084,3.4935,1.5217,6.5522,3.1739,11.4502,3.8238,8.7218,0.0000,2.5153
3,1967-03-02,4.6032,6.5820,3.4751,0,4.4700,1.0613,1.7011,3.4935,1.5326,6.5860,3.1772,11.4869,3.8177,8.7185,0.0100,2.5153
4,1967-03-09,4.6042,6.5852,3.4751,0,4.4700,1.0613,1.7011,3.4935,1.5326,6.5860,3.1772,11.4869,3.8177,8.7185,0.0100,2.5153


In [9]:
# difference transformation
# df = df.diff()
# df = df.drop(index='1967-02-09')

In [10]:
# Histograms of features
features = ['BCI', 'BCIp', 'BCIg']
features.extend(new_features)

def plot():
    for feature in features:
        df.hist(column = feature, bins = 50)
        plt.xlabel(feature,fontsize=15)
        plt.ylabel("Frequency",fontsize=15)
        plt.show()
        df.plot(y=feature)

# plot()

In [11]:
df.to_csv("../merged_data/features_USRECD.csv")